In [127]:
import pandas as pd
import requests
import re
import time

In [5]:
r = requests.get('https://zimas.lacity.org/map.aspx?pin=150A187   366&ajax=yes')

In [59]:
r.text

'{Address: "1769 N IVAR AVE", Addresses: ["1769 N IVAR AVE"] , selectedPins: [{pin: \'150A187   366\', addresses: ["1769 N IVAR AVE"] }], divTab1: "<table width=\\"100%\\" cellpadding=\\"0\\" cellspacing=\\"0\\" style=\'\'\\><tbody\\><tr\\><td class=\\"DataCellsLeft\\"\\><a style=\'cursor: pointer;\' onclick=\\"getToolTip(\'address.htm\');\\"\\>Site Address&nbsp;</a\\></td\\><td class=\\"DataCellsRight\\" \\>1769 N IVAR AVE</td\\></tr\\><tr\\><td class=\\"DataCellsLeft\\"\\><a style=\'cursor: pointer;\' onclick=\\"getToolTip(\'zipcode.htm\');\\"\\>ZIP Code&nbsp;</a\\></td\\><td class=\\"DataCellsRight\\" \\><a  href=\\"javascript:;\\" onclick=\\"ZimasData.openDataLink(\'ZIP_CD\', \'90028\');\\"\\>90028</a\\></td\\></tr\\><tr\\><td class=\\"DataCellsLeft\\"\\><a style=\'cursor: pointer;\' onclick=\\"getToolTip(\'pinnumber.htm\');\\"\\>PIN Number&nbsp;</a\\></td\\><td class=\\"DataCellsRight\\" \\>150A187&nbsp;&nbsp;&nbsp;366</td\\></tr\\><tr\\><td class=\\"DataCellsLeft\\"\\><a style=\'

In [140]:
lot_parcel_area_regex = 'Lot/Parcel Area \(Calculated\)&nbsp;</a\\\\></td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>([0123456789,.]+) \(sq ft\)'

community_plan_area_regex = 'Community Plan Area&nbsp;</a\\\\></td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\><a  href=\\\\"javascript:;\\\\" onclick=\\\\"ZimasData\.openDataLink\(\\\'COMN\\\', \\\'([^\\\\]+)\\\'\)'
area_planning_commission_regex = 'Area Planning Commission&nbsp;</a\\\\></td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\><a  href=\\\\"javascript:;\\\\" onclick=\\\\"ZimasData\.openDataLink\(\\\'APC\\\', \\\'([^\\\\]+)\\\'\)'
neighborhood_council_regex = 'Neighborhood Council&nbsp;</a\\\\></td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\><a  href=\\\\"javascript:;\\\\" onclick=\\\\"ZimasData\.openDataLink\(\\\'NC\\\', \\\'([^\\\\]+)\\\'\)'
council_district_regex = 'Council District&nbsp;</a\\\\></td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\><a  href=\\\\"javascript:;\\\\" onclick=\\\\"ZimasData\.openDataLink\(\\\'COUNCIL\\\', \\\'([^\\\\]+)\\\'\)'

assessed_land_value_regex = 'Assessed Land Val\.&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>\$([0123456789,]+)'
assessed_improvement_value_regex = 'Assessed Improvement Val\.&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>\$([0123456789,.]+)'
last_owner_change_regex = 'Last Owner Change&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>([0123456789/]+)'
last_sale_amount_regex = 'Last Sale Amount&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>\$([0123456789,.]+)'


year_built_regex = ('Year Built&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>(\d+)')
building_square_footage_regex = 'Building Square Footage&nbsp;</td\\\\><td class=\\\\"DataCellsRight\\\\" \\\\>([0123456789,.]+) \(sq ft\)'



In [66]:
df = pd.read_csv('Addresses_in_the_City_of_Los_Angeles.csv')

In [70]:
one_per_zip = df.groupby('ZIP_CD').first()

In [141]:
lot_parcel_areas = []
community_plan_areas = []
area_planning_commissions = []
neighborhood_councils = []
council_districts = []

assessed_land_values = []
assessed_improvement_values = []
last_owner_changes = []
last_sale_amounts = []

years_built = []
building_square_footages = []

start_time = time.time()
for (i,PIN) in enumerate(one_per_zip['PIN']):
    print(i)
    print(PIN)
    r = requests.get('https://zimas.lacity.org/map.aspx?pin='+PIN+'&ajax=yes')
    text_data = r.text
    
    lot_parcel_areas.append(float(re.search(lot_parcel_area_regex,text_data).group(1).replace(",", "")))
    try: 
        community_plan_areas.append(re.search(community_plan_area_regex,text_data).group(1))
    except:
        community_plan_areas.append('None')
    try:
        area_planning_commissions.append(re.search(area_planning_commission_regex,text_data).group(1))
    except:
        area_planning_commissions.append('None')
    try:
        neighborhood_councils.append(re.search(neighborhood_council_regex,text_data).group(1))
    except:
        neighborhood_councils.append('None')
    try:
        council_districts.append(re.search(council_district_regex,text_data).group(1))
    except:
        council_districts.append('None')
    try:
        assessed_land_values.append(float(re.search(assessed_land_value_regex,text_data).group(1).replace(",", "")))
    except:
        assessed_land_values.append('None')
    try:
        assessed_improvement_values.append(float(re.search(assessed_improvement_value_regex,text_data).group(1).replace(",", "")))
    except:
        assessed_improvement_values.append('None')
    try:
        last_owner_changes.append(re.search(last_owner_change_regex,text_data).group(1))
    except:
        last_owner_changes.append('None')
    try:
        last_sale_amounts.append(float(re.search(last_sale_amount_regex,text_data).group(1).replace(",", "")))
    except:
        last_sale_amounts.append('None')
    
    year_built_list = list(filter(lambda x: x!='0',re.findall(year_built_regex,text_data)))
    building_square_footage_list = list(filter(lambda x: x!='0',re.findall(building_square_footage_regex,text_data)))
    
    if year_built_list == []:
        years_built.append(0)
    else:
        years_built.append(min([int(x) for x in year_built_list]))
    if building_square_footage_list == []:
        building_square_footages.append(0)
    else:
        building_square_footages.append(sum([float(x.replace(",", "")) for x in building_square_footage_list]))
end_time = time.time()

    

0
097-5A207 217
1
097-5A207 386
2
099B201  1288
3
141B197   880
4
132B193   641
5
129A201   286
6
123B197   612
7
114B181   105
8
132B193    20
9
108B213   259
10
133-5A213 150


KeyboardInterrupt: 

In [118]:
one_per_zip['lot_parcel_area'] = lot_parcel_areas
one_per_zip['community_plan_area'] = community_plan_areas
one_per_zip['area_planning_commission'] = area_planning_commissions
one_per_zip['neighborhood_council'] = neighborhood_councils
one_per_zip['council_district'] = council_districts

one_per_zip['assessed_land_value'] = assessed_land_values
one_per_zip['assessed_improvement_values'] = assessed_improvement_values
one_per_zip['last_owner_change'] = last_owner_changes
one_per_zip['last_sale_amount'] = last_sale_amounts

one_per_zip['year_built'] = years_built
one_per_zip['building_square_footage'] = building_square_footages

In [126]:
one_per_zip.loc[[type(x) == float for x in one_per_zip['last_sale_amount']]].sort_values('last_sale_amount',ascending=False).head(50)

,HSE_ID,PIN,PIND,HSE_NBR,HSE_FRAC_NBR,HSE_DIR_CD,STR_NM,STR_SFX_CD,STR_SFX_DIR_CD,UNIT_RANGE,...,community_plan_area,area_planning_commission,neighborhood_council,council_district,assessed_land_value,assessed_improvement_values,last_owner_change,last_sale_amount,years_built,building_square_footage
ZIP_CD,,,,,,,,,,,,,,,,,,,,,
90045,1042028,096B169 869,096B169-869,8802,1/2,S,BELFORD,AVE,WEST,1-18,...,Westchester - Playa del Rey,West Los Angeles,Westchester/Playa,CD 11 - Traci Park,5.99519e+06,3.49716e+07,04/03/2012,3.00003e+07,1949,33112.0
90023,1136362,121-5A221 114,121-5A221-114,1241,1/2,S,SOTO,ST,NaN,1-3,...,Boyle Heights,East Los Angeles,Boyle Heights,CD 14 - Kevin de León,6.59822e+06,2.00018e+07,12/21/2017,2.16002e+07,1965,93641.0
90014,1038187,129A211 210,129A211-210,627,1/2,S,BROADWAY,ST,NaN,1-103,...,Central City,Central,Downtown Los Angeles,CD 14 - Kevin de León,3.6265e+06,3.53485e+06,03/03/2015,6.30006e+06,1910,56096.0
90029,763221,147B197 1160,147B197-1160,1277,1/2,N,VERMONT,AVE,NaN,1-16,...,Hollywood,Central,East Hollywood,CD 13 - Hugo Soto-Martinez,2.65302e+06,105,09/15/2016,5.34105e+06,0,0.0
91364,1034784,171B097 625,171B097-625,22996,1/2,W,VENTURA,BLVD,NaN,1-2,...,Canoga Park - Winnetka - Woodland Hills - West...,South Valley,Woodland Hills-Warner Center,CD 3 - Bob Blumenfield,3.89675e+06,1.79891e+06,08/24/2015,5.15005e+06,1985,14256.0
91607,87660,174B165 696,174B165-696,12419,1/2,W,BURBANK,BLVD,NaN,1-23,...,North Hollywood - Valley Village,South Valley,Greater Valley Glen,CD 2 - Paul Krekorian,5.82134e+06,5.10680e+06,06/14/2013,4.61455e+06,1961,39041.0
91423,12859,162B157 194,162B157-194,13443,1/2,W,GALEWOOD,ST,NORTH,1-6,...,Sherman Oaks - Studio City - Toluca Lake - Cah...,South Valley,Sherman Oaks,CD 4 - Nithya Raman,2.85191e+06,1.27239e+06,04/23/2021,3.99504e+06,1940,0.0
90041,1037123,162A221 160,162A221-160,5047,1/2,N,EAGLE ROCK,BLVD,NaN,1-2,...,Northeast Los Angeles,East Los Angeles,Eagle Rock,CD 14 - Kevin de León,1.40577e+06,540101,09/21/2010,3.44503e+06,2005,2880.0
90008,1051125,114B181 105,114B181-105,4068,1/2,S,ABOURNE,ROAD,NaN,1-11,...,West Adams - Baldwin Hills - Leimert,South Los Angeles,Empowerment Congress West Area,CD 10 - Heather Hutt,690215,1.06684e+06,12/03/2019,3.29503e+06,1948,15406.0


In [129]:
elapsed_time = (end_time - start_time) / len(one_per_zip['PIN'])
elapsed_time

0.850248226626166

In [130]:
(len(df)*elapsed_time)/(24*60*60)

10.144268292198579

In [139]:
10000%10000

0